<a href="https://colab.research.google.com/github/MikeCorv/llm_evals/blob/main/FinalRuns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q lm_eval[vllm,gpu] accelerate
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.1 MB/s eta 0:00:00
 

In [ ]:
import lm_eval
import torch
import bitsandbytes
import json
import gc

In [ ]:
gc.collect()

#Clear the cache
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Thu Dec 11 09:42:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   71C    P0             34W /   72W |     255MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
shots_list = [0,1,2,3,4,5]

In [ ]:
model_id_1 = "Qwen/Qwen2.5-3B-Instruct"
qwen_scores = []

In [ ]:
for shot_num in shots_list:
  results_qwen = lm_eval.simple_evaluate(
          model="hf",
          model_args=(
            f"pretrained={model_id_1},"
            "dtype=bfloat16,"
            "trust_remote_code=True,"
            "max_length=8192,"
            "attn_implementation=sdpa"
        ),
          tasks=["mmlu"],
          num_fewshot=shot_num,
          limit = 10,
          apply_chat_template=True,
          fewshot_as_multiturn=True,
          batch_size = 4
      )
  entry = {
              "model_id": model_id_1,
              "shots": shot_num,
              "results": results_qwen['results'] # This contains the total MMLU + all subtasks
          }
  qwen_scores.append(entry)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests: 100%|██████████| 2280/2280 [00:30<00:00, 74.47it/s] 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests: 100%|██████████| 2280/2280 [00:39<00:00, 57.31it/s] 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests: 100%|██████████| 2280/2280 [00:46<00:00, 48.57it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests: 100%|██████████| 2280/2280 [00:56<00:00, 40.58it/s]


In [ ]:
model_id_2 = "google/gemma-3-4b-it"
gemma_scores = []

In [ ]:
for shot_num in shots_list:
  results_gemma = lm_eval.simple_evaluate(
          model="hf",
          model_args=(
        f"pretrained=google/gemma-3-4b-it,"
        "dtype=bfloat16,"
        "trust_remote_code=True,"     # Required for new Gemma 3 architecture
        "max_length=8192,"
        "attn_implementation=eager"
    ),
          tasks=["mmlu"],
          num_fewshot=shot_num,
          limit = 10,
          apply_chat_template=True,
          fewshot_as_multiturn=True,
          batch_size = "auto"
      )
  entry = {
              "model_id": model_id_2,
              "shots": 5,
              "results": results_gemma['results'] # This contains the total MMLU + all subtasks
          }
  gemma_scores.append(entry)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests:   0%|          | 0/2280 [00:00<?, ?it/s]

Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 2


Running loglikelihood requests: 100%|██████████| 2280/2280 [02:46<00:00, 13.66it/s]


In [ ]:
model_id_3 = "meta-llama/Llama-3.2-3B-Instruct"
llama_scores = []

In [ ]:
for shot_num in shots_list:
  results_llama = lm_eval.simple_evaluate(
        model="hf",
        model_args=(
            f"pretrained={model_id_3},"
            "dtype=bfloat16,"
            "trust_remote_code=True,"
            "max_length=8192,"
            "attn_implementation=eager"
        ),
        tasks=["mmlu"],
        num_fewshot=shot_num,
        limit=10, # keeps it fast for testing
        apply_chat_template=True,
        fewshot_as_multiturn=True,

        batch_size="auto"
    )
  entry = {
                "model_id": model_id_3,
                "shots": shot_num,
                "results": results_llama['results'] # This contains the total MMLU + all subtasks
            }
  llama_scores.append(entry)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running loglikelihood requests:   0%|          | 0/2280 [00:00<?, ?it/s]

Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 3


Running loglikelihood requests: 100%|██████████| 2280/2280 [02:21<00:00, 16.12it/s]
